<a href="https://colab.research.google.com/github/JDVila/Accelerated_Parallel_Computing_Assignment_07/blob/main/Accelerated_Parallel_Computing_Assignment_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-bd3s9r8_
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-bd3s9r8_
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done


In [3]:
%load_ext nvcc_plugin

directory /content/src already exists
Out bin /content/result.out


In [4]:
%%cu
#include <stdio.h>

__global__
void global_max(int* values, int* gl_min, int* gl_max)
{
  int i = threadIdx.x + blockDim.x * blockIdx.x;
  int val = values[i];
  atomicMin(gl_min, val);
  atomicMax(gl_max, val);
}

int main(void)
{
  int N = 100;
  int *x, *y, *z, *d_x, *d_y, *d_z;
  x = (int*)malloc(N*sizeof(int));
  y = (int*)malloc(sizeof(int));
  z = (int*)malloc(sizeof(int));

  cudaMalloc(&d_x, N*sizeof(int));
  cudaMalloc(&d_y, sizeof(int));
  cudaMalloc(&d_z, sizeof(int));

  for (int i = 0; i < N; i++) {
    x[i] = i * (i%9 +1);
  }

  cudaMemcpy(d_x, x, N*sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(d_y, y, sizeof(int), cudaMemcpyHostToDevice);
  cudaMemcpy(d_z, z, sizeof(int), cudaMemcpyHostToDevice);

  global_max<<<1,N>>>(d_x, d_y, d_z);

  cudaMemcpy(y, d_y, sizeof(int), cudaMemcpyDeviceToHost);
  cudaMemcpy(z, d_z, sizeof(int), cudaMemcpyDeviceToHost);

  printf("gl_min: %i\n", *y);
  printf("gl_max: %i\n", *z);

  cudaFree(d_x);
  cudaFree(d_y);
  cudaFree(d_z);
  free(x);
  free(y);
  free(z);
}

gl_min: 0
gl_max: 882

